In [1]:
import requests
import json
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import time



/Users/kris/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
API_key = "I4RBGATDG75HV3u1kzYOwZzGjIlkbChePMp4Z7WJ"

#pool_query = 3687 #zkSync Maverlic WETH-USDC.e
#pool_query = 3687 #zkSync SyncSwap WETH-USDC.e
#contract_name = 'zkSwap-SyncSwap-WETH-USDCe'
#token0 = 'WETH'
#token1 = 'USDC'

#pool_query = 3683 #Ethereum Uniswap v3 USDC-WETH
#contract_name = 'ethereum-Uniswap-v3-WETH-USDC'
#token0 = 'USDC'
#token1 = 'WETH'

pool_query = 3665 #Arbitrum Uniswap v3 WETH-USD
contract_name = 'arbitrum-Uniswap-v3-WETH-USDC'
token0 = 'WETH'
token1 = 'USDC'

#pool_query = 3685 #Arbitrum Uniswap v3 WETH-USDCe
#contract_name = 'arbitrum-Uniswap-v3-WETH-USDCe'
#token0 = 'WETH'
#token1 = 'USDC'

#pool_query = 3702 #Optimism Uniswap v3 WETH-USD
#contract_name = 'optimism-Uniswap-v3-WETH-USDC'
#token0 = 'WETH'
#token1 = 'USDC'

#pool_query = 3701 #BASE Uniswap v3 WETH-USD
#contract_name = 'base-Uniswap-v3-WETH-USDC'
#token0 = 'WETH'
#token1 = 'USDC'

#pool_query = 3758 #zkSync Uniswap v3 WETH-USD #3758 instead of 3709
#contract_name = 'zkSync-Uniswap-v3-WETH-USDC'
#token0 = 'USDC'
#token1 = 'WETH'



In [3]:

fee = 8  # Adjust the fee value accordingly
AMOUNT_USD = 'amount1'
AMOUNT_ETH = 'amount0'
RESERVE_USD = 'reserve_1'
RESERVE_ETH = 'reserve_0'
CONST_Token0 = 1e18 #WETH 
CONST_Token1 = 1e6 #USDC
CONST_Token0_Token1 = CONST_Token0 / CONST_Token1

if (token0 == 'USDC') :
    AMOUNT_USD = 'amount0'
    AMOUNT_ETH = 'amount1'
    RESERVE_USD = 'reserve_0'
    RESERVE_ETH = 'reserve_1'
    CONST_Token1 = 1e18 #WETH 
    CONST_Token0 = 1e6 #USDC
    CONST_Token0_Token1 = CONST_Token0 / CONST_Token1



In [4]:
def get_swaps_df(pool_query, API_key, offset=0, limit=100_000):
    query_url = f"https://query.nansen.ai/api/queries/{pool_query}/results?api_key={API_key}"
    json_data = {
        "parameters": {
            "limit": str(limit),
            "offset": str(offset),
        },
        "max_age": 0,
    }

    # Send the initial query request
    response = requests.post(query_url, json=json_data)
    if response.status_code != 200:
        print(f"Error initiating query: {response.text}")
        return None

    response_data = response.json()
    print("Initial response:", response_data)

    job_id = response_data.get("job", {}).get("id")
    job_url = f"https://query.nansen.ai/api/jobs/{job_id}?api_key={API_key}"

    query_result_id = None
    while True and job_id is not None:
        job_response = requests.get(job_url)
        if job_response.status_code != 200:
            print("Error getting job")
            break
        job_data = job_response.json()
        job = job_data.get("job")
        if job.get("status") == 3:
            print("Job completed")
            query_result_id = job.get("query_result_id")
            break
        elif job.get("status") in [4, 5]:
            print(job)
            print("Job failed")
            break
        else:
            print("Job still running")
            time.sleep(30)

    if query_result_id is None:
        print("No query result id")
        return None

    query_response_url = f"https://query.nansen.ai/api/queries/{pool_query}/results/{query_result_id}.json?api_key={API_key}"
    query_response = requests.get(query_response_url)

    if query_response.status_code == 200:
        try:
            query_data = query_response.json()
        except json.JSONDecodeError as e:
            print(f"JSON decode error: {e.msg}")
            print(f"Query response content: {query_response.text}")
            return None
    else:
        print(f"Failed to retrieve query data: {query_response.status_code}")
        print(f"Response content: {query_response.text}")
        return None

    # Process the query data into a Polars DataFrame
    query_result = query_data['query_result']
    columns = query_result['data']['columns']
    rows = query_result['data']['rows']

    # Create a dictionary for constructing the Polars DataFrame
    data_dict = {column['name']: [row[column['name']] for row in rows] for column in columns}

    # Create the Polars DataFrame
    swaps_df = pl.DataFrame(data_dict)

    # Convert block_timestamp column to datetime type
    if 'block_timestamp' in swaps_df.columns:
        swaps_df = swaps_df.with_columns(
            pl.col("block_timestamp").str.to_datetime("%Y-%m-%dT%H:%M:%S")
        )

    return swaps_df



In [5]:
# Example usage:
offset = 0
limit = 100_000

swaps_df = get_swaps_df(pool_query, API_key, offset, limit)
swaps_df_all = swaps_df
n = swaps_df.shape[0]
del swaps_df
while ( n == limit) :
    offset = offset + limit
    swaps_df = get_swaps_df(pool_query, API_key, offset, limit)
    swaps_df_all = swaps_df_all.vstack(swaps_df)
    n = swaps_df.shape[0]
    del swaps_df

if swaps_df_all is not None:
    print(swaps_df_all)


Initial response: {'job': {'id': 'f7eaebbe-973c-4cc3-b11c-c92b820420f0', 'updated_at': 0, 'status': 1, 'error': '', 'result': None, 'query_result_id': None}}
Job still running
Job still running
Job completed
Initial response: {'job': {'id': '33ad81c4-db9d-48a2-ba24-1235e02f5d52', 'updated_at': 0, 'status': 1, 'error': '', 'result': None, 'query_result_id': None}}
Job still running
Job still running
Job completed
Initial response: {'job': {'id': '34b37f1f-fabc-45c9-a0b4-b9b1db76743f', 'updated_at': 0, 'status': 1, 'error': '', 'result': None, 'query_result_id': None}}
Job still running
Job still running
Job completed
Initial response: {'job': {'id': '1fe78dfb-cb1d-49d5-a829-d030b82ebe0d', 'updated_at': 0, 'status': 1, 'error': '', 'result': None, 'query_result_id': None}}
Job still running
Job still running
Job completed
Initial response: {'job': {'id': '610c5a2e-ea5f-450d-8764-1edd90b87aea', 'updated_at': 0, 'status': 1, 'error': '', 'result': None, 'query_result_id': None}}
Job still 

AttributeError: 'NoneType' object has no attribute '_df'

In [7]:
# Specify the path to save the Parquet file
parquet_file_path = "./data/raw-data-"+contract_name+".paraquet"

# Write the DataFrame to a Parquet file
swaps_df_all.write_parquet(parquet_file_path)


In [6]:
swaps_df_all


transaction_hash,transaction_index,log_index,block_number,block_timestamp,topics,data,gas_price,receipt_gas_used
str,i64,i64,i64,datetime[μs],list[str],str,i64,i64
"""0x23e32f37d8f5…",1,7,206540031,2024-04-30 23:59:58,"[""0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67"", ""0x00000000000000000000000000000000ede6d8d217c60f93191c060747324bca"", ""0x00000000000000000000000000000000ede6d8d217c60f93191c060747324bca""]","""0xffffffffffff…",10000000,383585
"""0x554d981ac719…",4,29,206539987,2024-04-30 23:59:47,"[""0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67"", ""0x00000000000000000000000000000000ede6d8d217c60f93191c060747324bca"", ""0x00000000000000000000000000000000ede6d8d217c60f93191c060747324bca""]","""0x000000000000…",10000000,644787
"""0x2d9594f68cce…",6,48,206539987,2024-04-30 23:59:47,"[""0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67"", ""0x00000000000000000000000060ca5def792e7528f8ff48b7aea60c1e2234294b"", ""0x00000000000000000000000060ca5def792e7528f8ff48b7aea60c1e2234294b""]","""0xffffffffffff…",911000000,334915
"""0x453e895d01f5…",2,12,206539982,2024-04-30 23:59:46,"[""0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67"", ""0x00000000000000000000000000000000ede6d8d217c60f93191c060747324bca"", ""0x00000000000000000000000000000000ede6d8d217c60f93191c060747324bca""]","""0xffffffffffff…",10000000,664519
"""0xa860bcc1cd6c…",2,12,206539975,2024-04-30 23:59:44,"[""0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67"", ""0x00000000000000000000000000000000ede6d8d217c60f93191c060747324bca"", ""0x00000000000000000000000000000000ede6d8d217c60f93191c060747324bca""]","""0x000000000000…",10000000,675561
…,…,…,…,…,…,…,…,…
"""0x32154203c849…",1,39,187373660,2024-03-05 15:08:11,"[""0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67"", ""0x000000000000000000000000f111ed85e40931938bf0278e26176caa088e7cac"", ""0x000000000000000000000000f111ed85e40931938bf0278e26176caa088e7cac""]","""0x000000000000…",3148210000,5341191
"""0x4cdcaab1c1d9…",2,4,187373643,2024-03-05 15:08:07,"[""0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67"", ""0x0000000000000000000000006a3e4b7e23661108aaec70266c468e6c679ae022"", ""0x00000000000000000000000044c6c4c7da174f47ac81e818d55c02600390e79d""]","""0xffffffffffff…",170080000,1347679
"""0x708d32814bf8…",3,9,187373633,2024-03-05 15:08:04,"[""0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67"", ""0x00000000000000000000000068b3465833fb72a70ecdf485e0e4c7bd8665fc45"", ""0x0000000000000000000000004700192f8a4a00f009d87a515ff2d13e5cab8364""]","""0xffffffffffff…",2000000000,2909644


In [8]:


unique = swaps_df_all['transaction_hash'].n_unique()
block_numbers = swaps_df_all['block_number'].n_unique()
minBlock = swaps_df_all['block_number'].min()
maxBlock = swaps_df_all['block_number'].max()
avgInBlock = unique / block_numbers

my_row = f"\\num{{{unique}}} & \\num{{{block_numbers}}} & \\num{{{minBlock}}}--\\num{{{maxBlock}}} & \\num{{{avgInBlock:.2f}}}"
print(my_row)


\num{2367361} & \num{1709619} & \num{187373628}--\num{206540031} & \num{1.38}


In [9]:
avgInBlock f gjg

SyntaxError: invalid syntax (1766795497.py, line 1)

In [ ]:
# Specify the path to save the Parquet file
parquet_file_path = "./data/raw-data-"+contract_name+".paraquet"

# Write the DataFrame to a Parquet file
swaps_df.write_parquet(parquet_file_path)



In [ ]:
my_query = 'some SQL query OFFSET {};'
data = []
for counter in range(OFFSETs):
    response = requests.get/post(my_query.format(counter))
    data.append(response)
df = pandas.DataFrame(data)